# SSMR: generate SSM model from trajectory data

In [ ]:
import numpy as np
from scipy.io import loadmat


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import plot_utils as plot
import matplotlib.pyplot as plt
import utils as utils


## Import and inspect data
See mat file in URL

In [ ]:
Data = {}

In [ ]:
dataDecayObs = loadmat("../dataDecayObs.mat", simplify_cells=True)

Data['oData'] = dataDecayObs['oData']
Data['yData'] = dataDecayObs['yData']

Xsnapshots = dataDecayObs['Xsnapshots']

H = np.array(dataDecayObs['H'])
V = np.array(dataDecayObs['V'])
M = np.array(dataDecayObs['M'])

# oData = dataDecayObs['oData']
# yData = dataDecayObs['yData']

rDOF = 3
oDOF = 3
nTRAJ = Data['oData'].shape[0]
nDOF = Xsnapshots.shape[0]//2

In [ ]:
print(Data['oData'].shape)
print(Data['oData'][0, 1].shape)
print(Data['oData'][:, 1].shape)

### Plot trajectories in an observable of interest

In [ ]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 0+oDOF), ('oData', 2)],
             xyz_names=[r'$x$', r'$\dot{x}$', r'$z$'])

### Show PCA infos
Modes were already computed to get yData

In [ ]:
# Perform SVD on displacement field
svd = np.linalg.svd
s = svd(Xsnapshots[0:nDOF, :], compute_uv=False)
l2vals = s**2

In [ ]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=9)

## Obtain reduced-order coordinates

In [ ]:
plt.close('all')
# plot trajectories in 3D [x1_bar, x1_bar_dot, y] space
plot.traj_3D(Data,
             xyz_idx=[('yData', 0), ('yData', 0+rDOF), ('oData', 1)],
             xyz_names=[r'$\bar{x}_1$', r'$\dot{\bar{x}}_1$', r'$y$'])
# plot evolution of x, y and z in time, separately in 3 subplots
plot.traj_xyz(Data,
              xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'])

### Train and test data (train/test split)

In [ ]:
indTest = [0, 2, 4, 6, 8, 10, 12, 14]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

### Truncate trajectories

In [ ]:
t_interval = [0.14, 2.6]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

In [ ]:
print(Data['oDataTrunc'][0, 0].shape)
print(Data['oDataTrunc'][0, 1].shape)

Visualize truncated trajectories

In [ ]:
plt.close('all')
plot.traj_3D(Data,
             xyz_idx=[('yDataTrunc', 0), ('yDataTrunc', 0+rDOF), ('oDataTrunc', 1)],
             xyz_names=[r'$\bar{x}_1$', r'$\dot{\bar{x}}_1$', r'$y$'])

In [ ]:
plt.close('all')

## Use Matlab engine to evoke SSMLearn

Start Matlab engine and install/run SSMLearn

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [ ]:
# make data ready for Matlab
oDataTruncTrain_matlab = [[], []]
yDataTruncTrain_matlab = [[], []]
for i in range(len(indTrain)):
    oDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['oDataTrunc'][indTrain, :][i, 0].tolist()))
    oDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['oDataTrunc'][indTrain, :][i, 1].tolist()))
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][indTrain, :][i, 0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][indTrain, :][i, 1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [ ]:
SSMDim = 2*rDOF
obsDim = 2*oDOF
SSMOrder = 3

IMInfo = eng.IMGeometry(oDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', yDataTruncTrain_matlab)
IMInfoInv = eng.IMGeometry(yDataTruncTrain_matlab, obsDim, SSMOrder,
                           'reducedCoordinates', oDataTruncTrain_matlab)


### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [ ]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(yDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default')
# RDInfo['eigenvaluesLinPartFlow'][0:3]
RDInfo['conjugateDynamics']['polynomialOrder']

Stop Matlab engine -- not needed anymore

In [ ]:
eng.quit()

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [ ]:
oRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [ ]:
oRec['Train'] = utils.lift_trajectories(IMInfo, Data['yDataTrunc'][indTrain, :])
errorGeo['Train'] = utils.compute_trajectory_errors(oRec['Train'], Data['oDataTrunc'][indTrain, :])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4f}")


#### Test error

In [ ]:
oRec['Test'] = utils.lift_trajectories(IMInfo, Data['yDataTrunc'][indTest, :])
errorGeo['Test'] = utils.compute_trajectory_errors(oRec['Test'], Data['oDataTrunc'][indTest, :])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4f}")

Plot comparison of SSM-predicted vs. actual test trajectories

In [ ]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(oRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### SSM geometry (chart)

In [ ]:
yRec = {}

#### Train error

In [ ]:
yRec['CTrain'] = utils.lift_trajectories(IMInfoInv, Data['oDataTrunc'][indTrain, :])
errorGeo['CTrain'] = utils.compute_trajectory_errors(yRec['CTrain'], Data['yDataTrunc'][indTrain, :])[0] * 100

meanErrorGeo['CTrain'] = np.mean(errorGeo['CTrain'])
print(f"Average chart train error: {meanErrorGeo['CTrain']:.4f}")

#### Test error

In [ ]:
yRec['CTest'] = utils.lift_trajectories(IMInfoInv, Data['oDataTrunc'][indTest, :])
errorGeo['CTest'] = utils.compute_trajectory_errors(yRec['CTest'], Data['yDataTrunc'][indTest, :])[0] * 100

meanErrorGeo['CTest'] = np.mean(errorGeo['CTest'])
print(f"Average chart test error: {meanErrorGeo['CTest']:.4f}")

Plot comparison of SSM-predicted vs. actual test trajectories

In [ ]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('CTest', 0), ('CTest', 1), ('CTest', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              axs=axs, ls=':', color='darkblue')

Store map in the same variables

In [ ]:
for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
    IMInfo['chart'][key] = IMInfoInv['parametrization'][key]

### Reduced dynamics

In [ ]:
yRec['RD'] = utils.advectRD(RDInfo, Data['yDataTrunc'])[0]
meanErrorDyn = {}

normedTrajDist = utils.compute_trajectory_errors(yRec['RD'], Data['yDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

In [ ]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [ ]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [ ]:
oRec['RDTrain'] = utils.lift_trajectories(IMInfo, yRec['RD'][indTrain, :])
errorGlo['Train'] = utils.compute_trajectory_errors(oRec['RDTrain'], Data['oDataTrunc'][indTrain, :])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

#### Test error

In [ ]:
oRec['RDTest'] = utils.lift_trajectories(IMInfo, yRec['RD'][indTest, :])
errorGlo['Test'] = utils.compute_trajectory_errors(oRec['RDTest'], Data['oDataTrunc'][indTest, :])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

In [ ]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(oRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Understand which mode is responsible for what motion

In [ ]:
outdofsMat = np.zeros((3, 2*nDOF))
output_node = 1354
outdofsPS = 3 * output_node + np.array([0, 1, 2])
for i in range(3):
    outdofsMat[i, outdofsPS[i]] = 1

redDynLinearPart = np.array(RDInfo['reducedDynamics']['coefficients'])[:, :2*rDOF]
redStiffnessMat = -redDynLinearPart[rDOF:, :rDOF]

w2, UconsRedDyn = np.linalg.eig(redStiffnessMat)
sorted_idx = np.argsort(w2)
consFreqs, UconsRedDyn = np.diag(np.sqrt(w2[sorted_idx])), UconsRedDyn[:, sorted_idx]

Ucons = np.kron(np.eye(2), UconsRedDyn)
Vcons = V @ Ucons

modesDir = outdofsMat @ Vcons[:, :rDOF]
modesFreq = np.abs(RDInfo['eigenvaluesLinPartFlow']).flatten()[:3]

plt.close('all')
plot.mode_direction(modesDir, modesFreq)

## Control

$H_{red} = V^TM^{-1}H$

In [ ]:
Hred = V[:nDOF, :rDOF].T @ np.linalg.solve(M, H)

normsModalForcing = np.mean(Hred**2, axis=0)
normsNonModalforcing = np.mean(((np.eye(nDOF) - V[:nDOF, :rDOF] @ V[:nDOF,:rDOF].T) @ np.linalg.solve(M, H)) ** 2, axis=0)

In [ ]:
normsModalForcing

In [ ]:
normsNonModalforcing

### Setup model for control

In [ ]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d 

In [ ]:
Rauton = utils.Rauton(RDInfo)
C = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])
W = lambda x: IMInfo['chart']['H'] @ utils.multivariate_polynomial(x, IMInfo['chart']['polynomialOrder'])

Small example

In [ ]:

uFun = lambda t: np.array([np.arctan(t)/np.pi*2, 0, 0, 0]).reshape(-1, 1)
R = lambda t, y: Rauton(np.atleast_2d(y)) + np.vstack([np.zeros((rDOF, Hred.shape[1])), Hred]) @ uFun(t)

tTraj = np.linspace(0, 15, 1000)
sol = solve_ivp(R,
                t_span=[tTraj[0], tTraj[-1]],
                t_eval=tTraj,
                y0=np.zeros(2*rDOF),
                method='LSODA',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
yTraj = sol.y
oTraj = C(yTraj)

plt.close('all')
plot.traj_xyz_txyz(tTraj,
                   oTraj[outdofs[0], :],
                   oTraj[outdofs[1], :],
                   oTraj[outdofs[2], :])

### Integrate the model with inputs

#### Small inputs

In [ ]:
# Load the controls and actual trajectories for figure8
uFigureEight = np.loadtxt("../../system_data/u_small.csv", delimiter=',')
z = np.loadtxt("../../system_data/z_small.csv", delimiter=',')
z_eq = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
z_eq = np.concatenate([z_eq, [0, 0, 0]])

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_span = np.linspace(0, T, N+1)
z_true = z[:, 3:].T

interpFun = interp1d(t_span, uFigureEight, axis=0, fill_value="extrapolate")
uFun = lambda t: interpFun(t).reshape(-1, 1)

sol = solve_ivp(R,
                t_span=[t_span[0], t_span[-1]],
                t_eval=t_span,
                y0=np.zeros(2*rDOF),
                method='LSODA',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
yTraj = sol.y
oTraj = (C(yTraj).T + z_eq).T

plt.close('all')
ax = plot.traj_3D_xyz(oTraj[outdofs[0], :],
                      oTraj[outdofs[1], :],
                      oTraj[outdofs[2], :], show=False)
plot.traj_3D_xyz(z_true[outdofs[0], :],
                 z_true[outdofs[1], :],
                 z_true[outdofs[2], :], color="tab:orange", ax=ax)


#### Big inputs

In [ ]:
# Load the controls and actual trajectories for figure8
uFigureEight = np.loadtxt("../../system_data/u_big.csv", delimiter=',')
z = np.loadtxt("../../system_data/z_big.csv", delimiter=',')
z_eq = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
z_eq = np.concatenate([z_eq, [0, 0, 0]])
z_eight_true = np.hstack([z[:, 3:], z[:, :3]]).T
z_eight_shift = (z_eight_true.T - z_eq).T
U_eight = uFigureEight.T

In [ ]:
plt.close('all')

### Learn control matrix $B$

Use figure8 trajectory to identify the impact of control, i.e. regress the control matrix $B$

In [ ]:
diamond_train_full = loadmat("../diamond_train_full.mat", simplify_cells=True)
y, u, t = diamond_train_full['y'], diamond_train_full['u'], diamond_train_full['t']

y = y.T
U = u.T
z_true_train = np.vstack([y[3:, :], y[:3, :]])
z_shift_train = (z_true_train.T - z_eq).T
t_span_train = np.hstack([t, t_span[1:] + t[-1]])

# # Add test trajectory to training dataset
# U = np.hstack([U, U_eight[:, 1:]])
# z_shift = np.hstack([z_shift, z_eight_shift[:, 1:]])

# Use only test trajectory as the training dataset --> should well overfit
# U = U_eight[:, 1:]
# z_shift_train = z_eight_shift[:, 1:]

Xbar = np.array(IMInfo['chart']['H']) @ utils.multivariate_polynomial(z_shift_train, IMInfo['chart']['polynomialOrder'])
dXbarDt = np.gradient(Xbar, dt, axis=1)[:, 3:-3]

U = U[:, 3:-3]
Xbar = Xbar[:, 3:-3]

dXbarDt_ROM = Rauton(Xbar)

Learn $B$ matrix

In [ ]:
from sklearn.linear_model import Ridge

polyUorder = 2

# X = np.vstack([U, Xbar])
X = utils.multivariate_polynomial(U, order=polyUorder)
print(X.shape)

ridgeModel = Ridge(alpha=0, fit_intercept=False)
# ridgeModel.fit(U.T, (dXbarDt - dXbarDt_ROM).T)
ridgeModel.fit(X.T, (dXbarDt - dXbarDt_ROM).T)
B_learn = ridgeModel.coef_

In [ ]:
# # home-brewed ridge regression
# l = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + l * np.eye(X.shape[0]), X.T @ y)

### Integrate moel with inputs (big amplitude) -- and using learned influence of control

In [ ]:
# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_span = np.linspace(0, T, N+1)
z_true = z[:, 3:].T


interpFun = interp1d(t_span, uFigureEight, axis=0, fill_value="extrapolate")
uFun = lambda t: interpFun(t).reshape(-1, 1)
def R(t, y):
    # print(y.shape)
    return Rauton(np.atleast_2d(y)) + B_learn @ utils.multivariate_polynomial(uFun(t), order=polyUorder) #  np.vstack([np.zeros((rDOF, Hred.shape[1])), Hred]) @ uFun(t) # np.vstack([uFun(t), y])

sol = solve_ivp(R,
                t_span=[t_span[0], t_span[-1]],
                t_eval=t_span,
                y0=np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
yTraj = sol.y
oTraj = (C(yTraj).T + z_eq).T

plt.close('all')
ax = plot.traj_3D_xyz(oTraj[outdofs[0], :],
                      oTraj[outdofs[1], :],
                      oTraj[outdofs[2], :], show=False)
ax = plot.traj_3D_xyz(z_true[outdofs[0], :],
                      z_true[outdofs[1], :],
                      z_true[outdofs[2], :], color="tab:orange", ax=ax, show=False)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.show()


In [ ]:
RMSE = np.sum(np.sqrt(np.mean((oTraj[0:3, :] - z_true)**2, axis=0))) / oTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")